https://huggingface.co/docs/transformers/en/model_doc/encoder-decoder

# Model

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, GenerationConfig
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, EarlyStoppingCallback
from datasets import Dataset, DatasetDict, load_metric
import evaluate
import torch
import os
from os import listdir
from os.path import isfile, join
import json
import re
import numpy as np
import pandas as pd

In [4]:
torch.cuda.is_available()

True

In [ ]:
# # max_split_size_mb nya bisa dikecilin klo GPU nya OOM
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

In [5]:
folder_data = "/kaggle/input/dataset-article2/"
train_set = pd.read_csv(folder_data+'final_train_set.csv')
valid_set = pd.read_csv(folder_data+'final_valid_set.csv')
test_set = pd.read_csv(folder_data+'final_test_set.csv')

train_set = train_set.sample(n=3000, random_state=88).reset_index(drop=True)
valid_set = valid_set.sample(n=1000, random_state=88).reset_index(drop=True)
test_set = test_set.sample(n=1000, random_state=88).reset_index(drop=True)

In [6]:
train_set.shape, valid_set.shape, test_set.shape

((3000, 2), (1000, 2), (1000, 2))

In [7]:
train_set.duplicated().sum(), valid_set.duplicated().sum(), test_set.duplicated().sum()

(0, 0, 0)

In [8]:
train_set.head()

,final_clean_article,combined_clean_summary
0,Manajer Arsenal Arsene Wenger mewanti-wanti Re...,Manajer Arsenal Arsene Wenger menegaskan ia ak...
1,Obafemi Martins menjadi bintang pada laga Newc...,Obafemi Martins memberi kontribusi besar saat ...
2,Kejaksaan Tinggi Bengkulu memeriksa Wali Kota ...,Wali Kota Bengkulu Chalik Effendi diperiksa Ke...
3,Markas Besar TNI memutasi 66 perwira tinggi da...,Alih tugas 66 perwira itu terdiri dari 47 perw...
4,Keputusan pemerintah menunda surat keputusan b...,Jemaah Ahmadiyah menggelar doa dan sujud syuku...


In [9]:
valid_set.head()

,final_clean_article,combined_clean_summary
0,Pemerintah masih terkesan setengah hati dalam ...,"Pengamat politik Andi Malarangeng menilai, keb..."
1,Kalangan pekerja hiburan malam melakukan unjuk...,Pekerja hiburan malam berunjuk rasa di DPRD DK...
2,Semua mata dan perhatian tengah terfokus pada ...,Sejak awal Fraksi Partai Persatuan Pembangunan...
3,Tim Terpadu Penanggulangan Penyalahgunaan Baha...,Sejumlah SPBU diduga telah menyelewengkan jata...
4,Ratusan wartawan media cetak dan elektronik be...,Wartawan dari berbagai media cetak dan elektro...


In [10]:
test_set.head()

,final_clean_article,combined_clean_summary
0,"Gara-gara menganiaya pembantu rumah tangga, Ny...","Yuli, warga Jalan Pemuda IV, Rawamangun, Jakar..."
1,Buku kumpulan lukisan karya almarhum Hendra Gu...,Koleksi lukisan almarhum Hendra Gunawan diterb...
2,"Dalam rangka menyambut bulan Ramadhan, SCTV mu...",Sinetron mini seri Kembang Padang Kelabu diper...
3,"Ismail Tiris, Salokon Risun, dan Korenus Intor...",Tiga anggota Tentara Pembebasan Nasional Papua...
4,Bank Indonesia (BI) harus segera mengisi kekos...,Kekosongan sejumlah posisi Deputi Gubernur Ban...


In [40]:
train_dataset = Dataset.from_pandas(train_set)
valid_dataset = Dataset.from_pandas(valid_set)
test_dataset = Dataset.from_pandas(test_set)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

## Define Model

In [82]:
tokenizer = T5Tokenizer.from_pretrained("panggi/t5-base-indonesian-summarization-cased")
# tokenizer.bos_token = tokenizer.cls_token
# tokenizer.eos_token = tokenizer.sep_token
model = T5ForConditionalGeneration.from_pretrained("panggi/t5-base-indonesian-summarization-cased")

In [64]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [51]:
prefix = ""
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["final_clean_article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["combined_clean_summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [52]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Training

In [83]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.03,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    # logging_dir='./logs',
    # logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="rouge2",
    greater_is_better=True,
)

In [81]:
rouge = evaluate.load('rouge')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = preds[0]

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Directly decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects newline-separated text
    decoded_preds = ["\n".join(decoded_pred.split()) for decoded_pred in decoded_preds]
    decoded_labels = ["\n".join(decoded_label.split()) for decoded_label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {k: round(v, 4) for k, v in result.items()}

In [84]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [85]:
torch.cuda.empty_cache()

In [86]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,2.529671,0.291900,0.140900,0.250700,0.291900
2,1.937500,2.508332,0.295100,0.142000,0.253300,0.294800
3,1.677800,2.545074,0.299900,0.146600,0.256500,0.299200
4,1.497300,2.557124,0.299600,0.146200,0.256500,0.299400
5,1.497300,2.592143,0.299400,0.146400,0.257200,0.299400
6,1.344400,2.607188,0.301900,0.147500,0.259300,0.301800
7,1.270300,2.642250,0.302900,0.148100,0.259200,0.302600
8,1.180500,2.660941,0.305000,0.149200,0.261000,0.304300
9,1.180500,2.685381,0.303200,0.147600,0.259400,0.302700
10,1.138800,2.683406,0.303500,0.148400,0.260200,0.303100


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=3750, training_loss=1.415733203125, metrics={'train_runtime': 3320.0553, 'train_samples_per_second': 9.036, 'train_steps_per_second': 1.129, 'total_flos': 1.39248926014464e+16, 'train_loss': 1.415733203125, 'epoch': 10.0})

In [87]:
eval_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
print(eval_results)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 2.3925890922546387, 'eval_rouge1': 0.3323, 'eval_rouge2': 0.1824, 'eval_rougeL': 0.2869, 'eval_rougeLsum': 0.3326, 'eval_runtime': 126.7887, 'eval_samples_per_second': 7.887, 'eval_steps_per_second': 1.972, 'epoch': 10.0}


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [88]:
# Save the model
model.save_pretrained("/kaggle/working/t5_indo_sum")

# Save the tokenizer
tokenizer.save_pretrained("/kaggle/working/t5_indo_sum/tokenizer") 

('/kaggle/working/t5_indo_sum/tokenizer/tokenizer_config.json',
 '/kaggle/working/t5_indo_sum/tokenizer/special_tokens_map.json',
 '/kaggle/working/t5_indo_sum/tokenizer/spiece.model',
 '/kaggle/working/t5_indo_sum/tokenizer/added_tokens.json')

In [89]:
!zip -r t5_indo_sum.zip /kaggle/working/t5_indo_sum

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


updating: kaggle/working/t5_indo_sum/ (stored 0%)
updating: kaggle/working/t5_indo_sum/tokenizer/ (stored 0%)
updating: kaggle/working/t5_indo_sum/tokenizer/spiece.model (deflated 49%)
updating: kaggle/working/t5_indo_sum/tokenizer/special_tokens_map.json (deflated 85%)
updating: kaggle/working/t5_indo_sum/tokenizer/added_tokens.json (deflated 83%)
updating: kaggle/working/t5_indo_sum/tokenizer/tokenizer_config.json (deflated 94%)
updating: kaggle/working/t5_indo_sum/model.safetensors (deflated 7%)
updating: kaggle/working/t5_indo_sum/config.json (deflated 48%)
updating: kaggle/working/t5_indo_sum/generation_config.json (deflated 29%)


## Inference

In [ ]:
# # Load the tokenizer
# tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/model/bert2gpt_indo_sum/tokenizer")

# # Load the model
# model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/model/bert2gpt_indo_sum")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [90]:
def clean_article(text):
    # Remove phrases like "Liputan6.com, [City Name]:"
    text = re.sub(r'Liputan6\. com, [A-Za-z\s]+:', '', text)

    # Remove words followed by a date in the format (dd/mm) or (d/m)
    text = re.sub(r'\w+\s*\(\d{1,2}/\d{1,2}\)', '', text)

    # Remove text inside parentheses that follow a specific pattern (e.g., (UPI/Reporter Name)) and optionally with 'dan' conjunction
    text = re.sub(r'\([A-Z]+/[A-Za-z\s]+(?: dan [A-Za-z\s]+)?\)\.', '', text)

    # Remove text inside square brackets that starts with 'baca:'
    text = re.sub(r'\[baca: .*?\]', '', text)

    # Remove URLs starting with http or https
    text = re.sub(r'https?://\S+', '', text)

    # Remove leading and trailing whitespaces from the text
    return text.strip()

In [91]:
%%time
ARTICLE_TO_SUMMARIZE = """Bank Mandiri, sebagaimana bank umumnya, menyediakan layanan kartu debit bagi nasabahnya. Kartu debit Mandiri dapat digunakan oleh nasabah untuk melakukan berbagai transaksi di mesin ATM atau mesin EDC. Fungsi dari Kartu Debit Mandiri ini sangat beragam, mulai dari tarik tunai, setor tunai, transfer uang, cek saldo rekening, hingga membayar berbagai tagihan melalui mesin ATM.
Penting bagi Anda yang ingin membuka rekening tabungan di Bank Mandiri untuk memahami jenis Kartu Debit Mandiri agar tidak salah memilih. Setiap kartu debit Mandiri memiliki kelebihan dan kekurangannya masing-masing, sehingga penting bagi nasabah untuk memilih yang sesuai dengan kebutuhan dan preferensi mereka.

Dalam memilih jenis Kartu Debit Mandiri, nasabah perlu mempertimbangkan kebutuhan dan gaya hidup mereka. Apakah mereka membutuhkan manfaat tambahan seperti asuransi atau akses ke airport lounge, ataukah mereka menginginkan kartu debit yang sederhana namun praktis. Dengan mengetahui jenis Kartu Debit Mandiri yang sesuai, nasabah dapat memaksimalkan manfaat yang mereka dapatkan dari penggunaan kartu debit tersebut.
"""

# generate summary
input_ids = tokenizer.encode(clean_article(ARTICLE_TO_SUMMARIZE), return_tensors='pt')
summary_ids = model.generate(input_ids.to(model.device),
            min_length=20,
            max_length=128,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.1,
            top_k = 50,
            top_p = 0.95)
# start time dan end time
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

Kartu debit Mandiri dapat digunakan oleh nasabah untuk melakukan berbagai transaksi di mesin ATM atau mesin EDC.
CPU times: user 884 ms, sys: 36.5 ms, total: 920 ms
Wall time: 1.06 s


In [ ]:
# parent_folder = "/content/" # Pake yang ada di folder "clean_data" di Google drive
# test_set = pd.read_csv(parent_folder+'final_test_set.csv')

In [92]:
df_sample = test_set.sample(1)

In [93]:
df_sample["final_clean_article"].values

array(['Kepala Kepolisian Daerah Metro Jaya Inspektur Jenderal Polisi Sofjan Jacoeb menilai para pelaku kejahatan semakin nekat dan tak segan-segan melukai serta membunuh korban. Sebab itu, ia meminta masyarakat menggiatkan pengamanan swakarsa dan menjaga lingkungan masing-masing. Penilaian tersebut Sofjan sampaikan di Jakarta, baru-baru ini. Menurut Sofjan, musibah yang menimpa Brigadir Polisi Mursito, anggota Kepolisian Resor Jakarta Barat yang tewas saat mengawal uang nasabah Bank Central Asia, dapat menjadi cermin kebrutalan penjahat . Di samping itu, Sofjan menegaskan, insiden tersebut juga menjadi tantangan bagi kepolisian untuk memberikan rasa aman kepada masyarakat.'],
      dtype=object)

In [94]:
%%time
ARTICLE_TO_SUMMARIZE = """Kepala Kepolisian Daerah Metro Jaya Inspektur Jenderal Polisi Sofjan Jacoeb menilai para pelaku kejahatan semakin nekat dan tak segan-segan melukai serta membunuh korban. Sebab itu, ia meminta masyarakat menggiatkan pengamanan swakarsa dan menjaga lingkungan masing-masing. Penilaian tersebut Sofjan sampaikan di Jakarta, baru-baru ini. Menurut Sofjan, musibah yang menimpa Brigadir Polisi Mursito, anggota Kepolisian Resor Jakarta Barat yang tewas saat mengawal uang nasabah Bank Central Asia, dapat menjadi cermin kebrutalan penjahat . Di samping itu, Sofjan menegaskan, insiden tersebut juga menjadi tantangan bagi kepolisian untuk memberikan rasa aman kepada masyarakat."""

# generate summary
input_ids = tokenizer.encode(clean_article(ARTICLE_TO_SUMMARIZE), return_tensors='pt')
summary_ids = model.generate(input_ids.to(model.device),
            min_length=20,
            max_length=128,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.1,
            top_k = 50,
            top_p = 0.95)
# start time dan end time
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

Kepala Kepolisian Daerah Metro Jaya Inspektur Jenderal Polisi Sofjan Jacoeb menilai para pelaku kejahatan semakin nekat dan tak segan-segan melukai serta membunuh korban.
CPU times: user 1.32 s, sys: 3.95 ms, total: 1.32 s
Wall time: 1.32 s
